# task: train ALS models on the entire `ratings` dataset

* load ratings.csv file (provided)
* use ratings.csv to train a ALS model to provide recommendations
* evaluate model performance using RMSE. Possibly suggest other metrics, please justifiy if you use other metrics
* use GridSearch or other methods to adjust model Hyperparameters 
* comment on computational cost of optimisation, and the improvements achieved over the baseline model

## Data Loading

In [3]:
import pandas as pd
import pyspark.sql.functions as f

In [4]:
#IN 
RATINGS_SMALL_PARQUET = "/FileStore/tables/ratings-small.parquet"

In [5]:
ratings = spark.read.parquet(RATINGS_SMALL_PARQUET)
ratings.count()
ratings.cache()
display(ratings)

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041
1,157,5.0,964984100


In [6]:
# 1. Load ratings.csv file 

# File location and type
file_location = "/FileStore/tables/ratings.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
ratingss = spark.read.format(file_type)\
  .option("header", first_row_is_header)\
  .option("inferSchema", infer_schema)\
  .load(file_location)

ratingss.cache()
display(ratingss)

# rating = session.read.csv(path = RESOURCES+"ratings.csv", inferSchema=True, header=True, mode='DROPMALFORMED', multiLine=True)

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041
1,157,5.0,964984100


### here you write your ALS model training and evalutation code
Collaborative Filtering:  If two users Alice and Bob who give the similar ratings on the same movie, they will also have similar ratings on movies which they haven't seen before.

In [8]:
# 2. Use ratings to train a ALS model to provide recommendations
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.evaluation import RegressionEvaluator

# 2.1 Load and parse data
ratings = ratings.rdd.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratings)

# 2.2 Split the data set
(ratings_train, ratings_test) = ratings.randomSplit([0.8, 0.2])

# 2.3 Run the model
# rank = 10       # the number of latent factors in the model (defaults to 10).
maxIter = 5       # the maximum number of iterations to run (defaults to 10).
regParam = 0.01   # specifies the regularization parameter in ALS (defaults to 1.0).
als = ALS(maxIter=maxIter, regParam=regParam, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")  # "drop" in order to drop any rows in the DataFrame of predictions that contain NaN values. 

model = als.fit(ratings_train)

# 2.4 Predictions on the test set
predictions = model.transform(ratings_test)
display(predictions)

movieId,rating,timestamp,userId,prediction
471,3.0,874415126,372,3.981744
471,2.5,1498518822,599,3.152124
471,4.0,850766697,385,3.306603
471,2.5,1123890831,462,3.2761517
471,4.5,1238111129,104,3.5821636
471,3.0,856737165,32,4.352863
471,2.0,941558175,597,3.8122432
471,1.0,1005528017,500,2.31005
471,3.0,1139047519,387,3.0923014
471,3.0,975212641,216,3.5720778


In [9]:
# 3. Evaluate model performance using RSME
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print ("Root Mean Squared Error: " + str(rmse)) # 1.078233439523276

Root Mean Squared Error: 1.078233439523276

In [10]:
# 4. Use GridSearch or other methods to adjust model Hyperparameters
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# 4.1 ParamGridBuilder to construct a grid of parameters to search over
paramGrid = ParamGridBuilder()\
            .addGrid(als.rank, [1,5,10])\
            .addGrid(als.maxIter, [10,20])\
            .addGrid(als.regParam, [0.05,0.1,0.5])\
            .build()

# 4.2 Run cross-validation, and choose the best set of parameters.
crossval = CrossValidator(estimator=als, 
                          estimatorParamMaps=paramGrid, 
                          evaluator=evaluator, 
                          numFolds=3) # the number of folds for cross validation, >=2.

# 4.3 Train the model
cvModel = crossval.fit(ratings_train)   

# 4.4 Make predictions on test documents. cvModel uses the best model found (lrModel).
cvPrediction = cvModel.transform(ratings_test)
cvRMSE = evaluator.evaluate(cvPrediction)
print("Root Mean Square Error: " + str(cvRMSE)) # 0.8805003109654368
print ("Number of Models: ", len(paramGrid)) # 9

# 4.5 Best model
bestModel = cvModel.bestModel
print("rank: ", bestModel._java_obj.parent().getRank())  # add _java_obj.parent() to get parameters of the model.
print("maxIter: ", bestModel._java_obj.parent().getMaxIter())
print("regParam: ", bestModel._java_obj.parent().getRegParam())

java.net.SocketTimeoutException
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:118)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.rpc.JettyClient.withAttributionContext(JettyClient.scala:32)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:234)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:170)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:108)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:47)
	at com.databricks.backend.daemon.driver.DriverClient.com$databricks$backend$daemon$driver$DriverClient$$send(DriverClient.scala:95)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:283)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:89)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:299)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:245)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:299)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [11]:
# 5. Comment

1. Computational cost of optimisation

   It has 3 hyperparameters, and 18 models. The computational cost of optimisation is very high. Because the cost of a hyperparameter raises linearly, however the cost of multiple hyperparameters raise exponentially.

1. Improvements achieved over the baseline model

   Using deafault setting, rank = 10, maxIter = 15, regParam = 0.01, so the RMSE is 1.078233439523276

   After using grid search, rank = 5, maxIter = 20, regParam = 0.1, so the RMSE is 0.8802055613116014, with a decrease of 17.9%.

   As we can see, the hyperparameters chosen in the grid is very important, which can influence RMSE a lot. It has to try various hyperparameters to find the optimal model, and we have to spend a long time on big data.